In [4]:
from RiotAPI import *
from pprint import pprint

def get_timeline(game_id):
    api_data = requests.get("https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{0}?api_key={1}".format(game_id, api_key))
    json_data = json.loads(api_data.content.decode("utf-8"))
    return json_data

In [1]:
def get_map_image():
    
    map_image = Image.open("resource/map11.png", "r")
    map_image = map_image.resize((1500,1500))
    return np.asarray(map_image)

In [6]:
item_dict = get_item_json()
champion_dict = get_champion_json()

account_id = get_account_id("marwin")

# pprint(item_dict)

game_id_list = get_match_history(account_id)

data = get_timeline(game_id_list[0])

data2 = get_game_detail(game_id_list[0])
pprint(data2)

{'gameCreation': 1556896433528,
 'gameDuration': 854,
 'gameId': 3645823675,
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'gameVersion': '9.9.272.4950',
 'mapId': 11,
 'participantIdentities': [{'participantId': 1,
                            'player': {'accountId': 'AIh7d1Dk6l30nQ-QOrvK4h37oJpxNJ_v858x9qaUmPEtVXt8MIWjKqH7',
                                       'currentAccountId': 'AIh7d1Dk6l30nQ-QOrvK4h37oJpxNJ_v858x9qaUmPEtVXt8MIWjKqH7',
                                       'currentPlatformId': 'KR',
                                       'matchHistoryUri': '/v1/stats/player_history/KR/2188926292750208',
                                       'platformId': 'KR',
                                       'profileIcon': 3543,
                                       'summonerId': 'q1aOQljug-7VwGEG0FP4cD48pOjkr1g3Q8GR8CBEULRxdls',
                                       'summonerName': '낙성대사스케'}},
                           {'participantId': 2,
                            'player

                             'magicDamageDealtToChampions': 476,
                             'magicalDamageTaken': 2310,
                             'neutralMinionsKilled': 0,
                             'neutralMinionsKilledEnemyJungle': 0,
                             'neutralMinionsKilledTeamJungle': 0,
                             'objectivePlayerScore': 0,
                             'participantId': 6,
                             'pentaKills': 0,
                             'perk0': 8005,
                             'perk0Var1': 199,
                             'perk0Var2': 122,
                             'perk0Var3': 77,
                             'perk1': 9111,
                             'perk1Var1': 50,
                             'perk1Var2': 40,
                             'perk1Var3': 0,
                             'perk2': 9104,
                             'perk2Var1': 0,
                             'perk2Var2': 0,
                             'perk2Var3

In [ ]:
game_champion = {}


for participant in data2['participants']:
    game_champion[participant['participantId']] = champion_dict[str(participant['championId'])]
    
pprint(game_champion)

In [ ]:
pprint(data)
events = [data['frames'][event]['events'] for event in range(15)]
positions = []
victims = []
kill_times = []
for event in events:
    for each_event in event:
        time = each_event['timestamp'] / 60000
        if each_event['type'] == 'ITEM_PURCHASED' and each_event['participantId'] == 5:
            item_name = item_dict[str(each_event['itemId'])]['name']
            print(f'{time:0.1f}분에 구매한 아이템 : {item_name}')        
        elif each_event['type'] == 'ITEM_DESTROYED' and each_event['participantId'] == 5:
            item_name = item_dict[str(each_event['itemId'])]['name']
            print(f'{time:0.1f}분에 사라진 아이템 : {item_name}')
        elif each_event['type'] == 'CHAMPION_KILL' and each_event['killerId'] == 5:
            position = each_event['position']
            victimId = each_event['victimId']
            champion = game_champion[victimId]
            positions.append(position)
            victims.append(champion)
            kill_times.append(time)
            
plt.figure(figsize=(15, 15))
plt.rc('font', family=font_name)
plt.scatter([position['x'] / 10 for position in positions],[(15000 - position['y'])/10 for position in positions], zorder=1, color='red', marker = 'x')
for i in range(len(victims)):    
    plt.text(positions[i]['x'] / 10, (15000 - positions[i]['y']) / 10, f'{victims[i]} kill time: {kill_times[i]:0.1f}', fontsize=15, color='white')
plt.imshow(get_map_image(), zorder=0)
plt.show()